In [1]:
import ast
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
def load_data(file_path: str) -> pd.DataFrame:
    """
    Load data from a CSV file into a pandas DataFrame.

    Parameters:
    file_path (str): The path to the CSV file.

    Returns:
    pd.DataFrame: DataFrame containing the loaded data.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")

    return pd.read_csv(file_path)

def load_action_data(file_paths: list[str]) -> pd.DataFrame:
    """
    Load action data from multiple CSV files into a single DataFrame.

    Parameters:
    file_paths (list[str]): List of paths to the CSV files.

    Returns:
    pd.DataFrame: DataFrame containing the combined action data.
    """
    data_frames = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        data_frames.append(df)

    return pd.concat(data_frames, ignore_index=True)

def load_player_data(file_path: str) -> pd.DataFrame:
    """
    Load player data from a CSV file.

    Args:
        file_path (str): The path to the CSV file.

    Returns:
        pd.DataFrame: The DataFrame containing the player data.
    """
    return pd.read_csv(file_path, converters={'root_scores': ast.literal_eval, 'best_child_scores': ast.literal_eval})

In [3]:
def calculate_win_rate_per_player_type(data: pd.DataFrame) -> dict:
    """
    Calculate the win rates for each player type.

    Parameters:
    data (pd.DataFrame): DataFrame containing the game data.

    Returns:
    dict: A dictionary with player types as keys and their win rates as values.
    """
    player_types = set()

    for i in range(4):
        player_type_col = f'player_{i}_type'
        player_types.add(data[player_type_col].unique()[0])

    win_rates = {}

    for player_type in player_types:
        player_wins = data[data['winner_type'] == player_type]['game_id'].nunique()
        total_runs = data['game_id'].nunique()
        win_rate = player_wins / total_runs if total_runs > 0 else 0
        win_rates[player_type] = win_rate

    return win_rates

def calculate_win_rate_per_starting_position(data: pd.DataFrame) -> dict:
    wins_per_starting_position = data[['winner', 'game_id']].drop_duplicates().groupby('winner').count().reset_index().rename(columns={'game_id': 'count'}).set_index('winner').to_dict()['count']
    total_runs = data['game_id'].nunique()
    win_rates = {position: wins / total_runs for position, wins in wins_per_starting_position.items()}
    return win_rates

### Win rate for single experiment

In [ ]:
experiment_name = 'MCTS_vs_random-10s'
data_path = f'/Users/moshalangerak/Documents/GitLab/risk-agent/data/experiments_results/{experiment_name}/game_level.csv'

In [ ]:
data = load_data(data_path)

In [ ]:
data.head()

In [ ]:
win_rates_per_player_type = calculate_win_rate_per_player_type(data)
win_rates_per_starting_position = calculate_win_rate_per_starting_position(data)

In [ ]:
def plot_win_rates_per_player_type(win_rates: dict, title='Win Rate by Player Type') -> None:
    """
    Plot the win rate by type from the provided DataFrame.

    Parameters:
    data (pd.DataFrame): DataFrame containing the data to plot.
    """
    plt.figure(figsize=(6, 4))
    bars = plt.bar(win_rates.keys(), win_rates.values(), color='skyblue')
    plt.title(title)
    plt.xlabel('Player Type')
    plt.ylabel('Win Rate')
    plt.xticks(rotation=45)
    plt.ylim(0, 1)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Add win rate values above bars
    for bar, (key, value) in zip(bars, win_rates.items()):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{value:.3f}', ha='center', va='bottom', fontsize=10)

    plt.tight_layout()
    plt.show()

def plot_win_rates_per_starting_position(win_rates: dict) -> None:
    """
    Plot the win rate by starting position from the provided DataFrame.

    Parameters:
    data (pd.DataFrame): DataFrame containing the data to plot.
    """
    plt.figure(figsize=(6, 4))
    bars = plt.bar(win_rates.keys(), win_rates.values(), color='lightgreen')
    plt.title('Win Rate by Starting Position')
    plt.xlabel('Starting Position')
    plt.ylabel('Win Rate')
    plt.xticks(rotation=45)
    plt.ylim(0, 1)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Add win rate values above bars
    for bar, (key, value) in zip(bars, win_rates.items()):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{value:.3f}', ha='center', va='bottom', fontsize=10)

    plt.tight_layout()
    plt.show()

In [ ]:
plot_win_rates_per_player_type(win_rates_per_player_type, title=f'Win Rate by Player Type - {experiment_name}')

In [ ]:
plot_win_rates_per_starting_position(win_rates_per_starting_position)

### Action distribution for single experiment

In [ ]:
experiment_name = 'MCTS_vs_random-4s'
data_path = f'/Users/moshalangerak/Documents/GitLab/risk-agent/data/experiments_results/{experiment_name}/action_level/'

In [ ]:
# get the list of CSV files in the directory
csv_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]

In [ ]:
data = load_action_data([os.path.join(data_path, f) for f in csv_files])

In [ ]:
data.head()

In [ ]:
def count_action_occurrences(data: pd.DataFrame) -> pd.Series:
    """
    Count the occurrences of each action in the DataFrame.

    Parameters:
    data (pd.DataFrame): DataFrame containing the action data.

    Returns:
    pd.Series: Series with actions as index and their counts as values.
    """
    return data['action_type'].value_counts()

In [ ]:
action_occurences = count_action_occurrences(data)

In [ ]:
action_occurences

In [ ]:
def plot_action_occurences(action_counts: pd.Series, title: str ='Action Occurrences') -> None:
    """
    Plot the occurrences of actions from the provided Series.

    Parameters:
    action_counts (pd.Series): Series containing action counts.
    """
    plt.figure(figsize=(6, 5))
    bars = plt.bar(action_counts.index, action_counts.values, color='coral')
    plt.title(title)
    plt.xlabel('Action Type')
    plt.ylabel('Occurrences')
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Add occurrence values above bars
    for bar, count in zip(bars, action_counts):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{count}', ha='center', va='bottom', fontsize=10)

    plt.tight_layout()
    plt.show()

In [ ]:
plot_action_occurences(action_occurences, title=f'Action Occurrences - {experiment_name}')

### Average action distribution per player type for a single experiment

In [ ]:
experiment_name = 'MCTS_vs_random-5s'

action_data_path = f'/Users/moshalangerak/Documents/GitLab/risk-agent/data/experiments_results/{experiment_name}/action_level/'

In [ ]:
action_csv_files = [f for f in os.listdir(action_data_path) if f.endswith('.csv')]

In [ ]:
action_data = load_action_data([os.path.join(action_data_path, f) for f in action_csv_files])

In [ ]:
def average_action_occurence_per_turn_per_player_type(data: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the average action occurrence per turn for each player type.

    Parameters:
    data (pd.DataFrame): DataFrame containing the action data.

    Returns:
    pd.DataFrame: DataFrame with player types as index and action types as columns, 
                 containing average occurrences per turn.
    """
    # First, count actions per turn per player type per action type
    action_counts = data.groupby(['game_id', 'turn', 'player_type', 'action_type']).size().reset_index(name='count')

    # Then calculate the average count per turn for each player type and action type
    result = action_counts.groupby(['player_type', 'action_type'])['count'].mean().reset_index()

    result['average_per_turn'] = result['count']  # Rename for clarity
    result = result.pivot(index='player_type', columns='action_type', values='average_per_turn').fillna(0)

    result['Total'] = result.sum(axis=1)

    # round all values to 1 decimal place
    result = result.round(1)

    return result

In [ ]:
result = average_action_occurence_per_turn_per_player_type(action_data)

In [ ]:
result

### Compare win rates across multiple experiments (plot)

In [ ]:
experiments_names = [
    'MCTS_vs_random-1s',
    'MCTS_vs_random-2s',
    'MCTS_vs_random-3s',
    'MCTS_vs_random-4s',
    'MCTS_vs_random-5s',
    # 'MCTS_vs_random-10s',
    # 'MCTS_vs_random-20s',
    'MCTS_vs_basic_heuristic-1s',
    'MCTS_vs_basic_heuristic-2s',
    'MCTS_vs_basic_heuristic-3s',
    'MCTS_vs_basic_heuristic-4s',
    'MCTS_vs_basic_heuristic-5s',
    'MCTS_vs_basic_evaluation-1s',
    'MCTS_vs_basic_evaluation-2s',
    'MCTS_vs_basic_evaluation-3s',
    'MCTS_vs_basic_evaluation-4s',
    'MCTS_vs_basic_evaluation-5s'
]

# Define the experiments with their names, player types, and think time parameters
experiments = [
    ('MCTS_vs_random-1s', 'Random', 1),
    ('MCTS_vs_random-2s', 'Random', 2),
    ('MCTS_vs_random-3s', 'Random', 3),
    ('MCTS_vs_random-4s', 'Random', 4),
    ('MCTS_vs_random-5s', 'Random', 5),
    ('MCTS_vs_random-10s', 'Random', 10),
    ('MCTS_vs_random-20s', 'Random', 20),
    ('MCTS_vs_basic_heuristic-1s', 'Basic Heuristic', 1),
    ('MCTS_vs_basic_heuristic-2s', 'Basic Heuristic', 2),
    ('MCTS_vs_basic_heuristic-3s', 'Basic Heuristic', 3),
    ('MCTS_vs_basic_heuristic-4s', 'Basic Heuristic', 4),
    ('MCTS_vs_basic_heuristic-5s', 'Basic Heuristic', 5),
    ('MCTS_vs_basic_evaluation-1s', 'Basic Evaluation', 1),
    ('MCTS_vs_basic_evaluation-2s', 'Basic Evaluation', 2),
    ('MCTS_vs_basic_evaluation-3s', 'Basic Evaluation', 3),
    ('MCTS_vs_basic_evaluation-4s', 'Basic Evaluation', 4),
    ('MCTS_vs_basic_evaluation-5s', 'Basic Evaluation', 5)
]

In [ ]:
win_rates_all_experiments = {}

for experiment in experiments_names:
    data_path = f'/Users/moshalangerak/Documents/GitLab/risk-agent/data/experiments_results/{experiment}/game_level.csv'
    data = load_data(data_path)
    win_rates_per_player_type = calculate_win_rate_per_player_type(data)
    win_rates_all_experiments[experiment] = win_rates_per_player_type

In [ ]:
win_rates_all_experiments

In [ ]:
def plot_win_rates_experiments(win_rates: dict[str, dict], experiments: list[tuple[str, str, int]], title: str = 'Win Rates Across Experiments') -> None:
    """
    Plot the win rates across different experiments.

    Parameters:
    win_rates (Dict[str, dict]): Dictionary with experiment names as keys and win rates as values.
    """
    # Process the data to create a DataFrame suitable for plotting
    plot_data = []
    for exp_name, opponent, think_time in experiments:
        # Get the MCTS player's win rate for the experiment, defaulting to 0 if not found
        mcts_win_rate = win_rates_all_experiments[exp_name].get('mcts', 0)
        plot_data.append({
            'opponent_type': opponent,
            'think_time': think_time,
            'mcts_win_rate': mcts_win_rate
        })

    df = pd.DataFrame(plot_data)

    # --- Create the Plot ---
    plt.figure(figsize=(7, 5))

    # Define colors for different opponents for better visual distinction
    opponent_colors = {
        'Random': 'dodgerblue',
        'Basic Heuristic': 'coral',
        'Basic Evaluation': 'lightgreen',
        'black': 'black'
    }

    # Plot a separate line for each opponent type
    for opponent, group in df.groupby('opponent_type'):
        # Sort by think time to ensure the line connects points correctly
        group = group.sort_values('think_time')
        plt.plot(group['think_time'], group['mcts_win_rate'],
                marker='o', linestyle='-', label=opponent,
                color=opponent_colors.get(opponent, 'black'),
                linewidth=2, markersize=8)

    plt.title(title, fontsize=16, pad=20)
    plt.xlabel('Think Time (seconds)', fontsize=12)
    plt.ylabel('MCTS Win Rate', fontsize=12)
    plt.legend(title='Opponent Type', fontsize=10)
    plt.ylim(0, 1.0)
    plt.xticks(np.arange(1, 21, 1)) # Ensure x-axis ticks are integers from 1 to 20
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)

    plt.tight_layout()
    plt.show()

In [ ]:
plot_win_rates_experiments(win_rates_all_experiments, experiments, title='MCTS Player Win Rate For Different Opponents and Think Times')

### Calculate all win rates

In [ ]:
experiments_names = [
    'MCTS_vs_random-1s',
    'MCTS_vs_random-2s',
    'MCTS_vs_random-3s',
    'MCTS_vs_random-4s',
    'MCTS_vs_random-5s',
    'MCTS_vs_random-2s-hp',
    'MCTS_vs_random-5s-hp',
    'MCTS_vs_random-2s-hp-rafa',
    'MCTS_vs_random-ib',
    'MCTS_vs_random-ib-hp',
    'MCTS_vs_random-ib-hp-rafa',
    'MCTS_vs_basic_heuristic-1s',
    'MCTS_vs_basic_heuristic-2s',
    'MCTS_vs_basic_heuristic-3s',
    'MCTS_vs_basic_heuristic-4s',
    'MCTS_vs_basic_heuristic-5s',
    'MCTS_vs_basic_heuristic-2s-hp',
    'MCTS_vs_basic_heuristic-5s-hp',
    'MCTS_vs_basic_heuristic-2s-hp-rafa',
    'MCTS_vs_basic_heuristic-ib',
    'MCTS_vs_basic_heuristic-ib-hp',
    'MCTS_vs_basic_heuristic-ib-hp-rafa',
    'MCTS_vs_basic_evaluation-1s',
    'MCTS_vs_basic_evaluation-2s',
    'MCTS_vs_basic_evaluation-3s',
    'MCTS_vs_basic_evaluation-4s',
    'MCTS_vs_basic_evaluation-5s',
    'MCTS_vs_basic_evaluation-2s-hp',
    'MCTS_vs_basic_evaluation-5s-hp',
    'MCTS_vs_basic_evaluation-2s-hp-rafa',
    'MCTS_vs_basic_evaluation-ib',
    'MCTS_vs_basic_evaluation-ib-hp',
    'MCTS_vs_basic_evaluation-ib-hp-rafa',
]

In [ ]:
win_rates_all_experiments = {}

for experiment in experiments_names:
    data_path = f'/Users/moshalangerak/Documents/GitLab/risk-agent/data/experiments_results/{experiment}/game_level.csv'
    data = load_data(data_path)
    win_rates_per_player_type = calculate_win_rate_per_player_type(data)
    win_rates_all_experiments[experiment] = win_rates_per_player_type

win_rates_all_experiments

### Analyse number of iterations and depth reached by MCTS player

In [ ]:
experiments_names = [
    # 'MCTS_vs_random-1s',
    # 'MCTS_vs_random-2s',
    # 'MCTS_vs_random-3s',
    # 'MCTS_vs_random-4s',
    # 'MCTS_vs_random-5s',
    # 'MCTS_vs_random-2s-hp',
    # 'MCTS_vs_random-5s-hp',
    # 'MCTS_vs_random-2s-hp-rafa',
    # 'MCTS_vs_random-10s',
    'MCTS_vs_random-20s',
    # 'MCTS_vs_random-ib',
    # 'MCTS_vs_random-ib-hp',
    # 'MCTS_vs_random-ib-hp-rafa',
    # 'MCTS_vs_basic_heuristic-1s',
    # 'MCTS_vs_basic_heuristic-2s',
    # 'MCTS_vs_basic_heuristic-3s',
    # 'MCTS_vs_basic_heuristic-4s',
    # 'MCTS_vs_basic_heuristic-5s',
    # 'MCTS_vs_basic_heuristic-2s-hp',
    # 'MCTS_vs_basic_heuristic-5s-hp',
    # 'MCTS_vs_basic_heuristic-2s-hp-rafa',
    # 'MCTS_vs_basic_heuristic-ib',
    # 'MCTS_vs_basic_heuristic-ib-hp',
    # 'MCTS_vs_basic_heuristic-ib-hp-rafa',
    # 'MCTS_vs_basic_evaluation-1s',
    # 'MCTS_vs_basic_evaluation-2s',
    # 'MCTS_vs_basic_evaluation-3s',
    # 'MCTS_vs_basic_evaluation-4s',
    # 'MCTS_vs_basic_evaluation-5s',
    # 'MCTS_vs_basic_evaluation-2s-hp',
    # 'MCTS_vs_basic_evaluation-5s-hp',
    # 'MCTS_vs_basic_evaluation-2s-hp-rafa',
    # 'MCTS_vs_basic_evaluation-ib',
    # 'MCTS_vs_basic_evaluation-ib-hp',
    # 'MCTS_vs_basic_evaluation-ib-hp-rafa',
]

In [ ]:
experiments_names = [
    'MCTS_vs_random-1s',
    'MCTS_vs_random-2s',
    'MCTS_vs_random-3s',
    'MCTS_vs_random-4s',
    'MCTS_vs_random-5s',
    'MCTS_vs_basic_heuristic-1s',
    'MCTS_vs_basic_heuristic-2s',
    'MCTS_vs_basic_heuristic-3s',
    'MCTS_vs_basic_heuristic-4s',
    'MCTS_vs_basic_heuristic-5s',
]

In [ ]:
def calculate_average_root_visit_count(data: pd.DataFrame) -> float:
    """
    Calculate the average number of root visits from the MCTS player data.

    Args:
        data (pd.DataFrame): The DataFrame containing MCTS player data.

    Returns:
        float: The average number of root visits.
    """
    return data['root_visit_count'].mean()

def calculate_average_max_depth_reached(data: pd.DataFrame) -> float:
    """
    Calculate the average maximum depth reached from the MCTS player data.

    Args:
        data (pd.DataFrame): The DataFrame containing MCTS player data.

    Returns:
        float: The average maximum depth reached.
    """
    return data['max_depth_reached'].mean()

def calculate_median_max_depth_reached(data: pd.DataFrame) -> float:
    """
    Calculate the median maximum depth reached from the MCTS player data.

    Args:
        data (pd.DataFrame): The DataFrame containing MCTS player data.
    
    Returns:
        float: The median maximum depth reached.
    """
    return data['max_depth_reached'].median()

def calculate_max_max_depth_reached(data: pd.DataFrame) -> float:
    """
    Calculate the maximum depth reached from the MCTS player data.

    Args:
        data (pd.DataFrame): The DataFrame containing MCTS player data.
    
    Returns:
        float: The maximum depth reached.
    """
    return data['max_depth_reached'].max()

In [ ]:
root_visit_counts_per_experiment = {}
avg_max_depth_reached_per_experiment = {}
med_max_depth_reached_per_experiment = {}
max_max_depth_reached_per_experiment = {}

for experiment in experiments_names:
    data_path = os.listdir(f'/Users/moshalangerak/Documents/GitLab/risk-agent/data/experiments_results/{experiment}/player_level/')

    root_visit_counts = []
    avg_max_depths = []
    med_max_depths = []
    max_max_depths = []

    for file in data_path:
        if not file.endswith('.csv'):
            continue

        file = os.path.join(f'/Users/moshalangerak/Documents/GitLab/risk-agent/data/experiments_results/{experiment}/player_level/', file)
        print(file)
        data = load_player_data(file)
        root_visit_count = calculate_average_root_visit_count(data)
        root_visit_counts.append(root_visit_count)

        avg_max_depth = calculate_average_max_depth_reached(data)
        avg_max_depths.append(avg_max_depth)

        med_max_depth = calculate_median_max_depth_reached(data)
        med_max_depths.append(med_max_depth)

        max_max_depth = calculate_max_max_depth_reached(data)
        max_max_depths.append(max_max_depth)

    root_visit_counts_per_experiment[experiment] = np.mean(root_visit_counts)
    avg_max_depth_reached_per_experiment[experiment] = np.mean(avg_max_depths)
    med_max_depth_reached_per_experiment[experiment] = np.median(med_max_depths)
    max_max_depth_reached_per_experiment[experiment] = np.max(max_max_depths)

In [ ]:
root_visit_counts_per_experiment

In [ ]:
avg_max_depth_reached_per_experiment

In [ ]:
med_max_depth_reached_per_experiment

In [ ]:
max_max_depth_reached_per_experiment

### Win rates for tuning experiments

In [4]:
tuning_experiment = 'Paranoid_fortify_all_tuning'
tuning_experiments_folder = f'/Users/moshalangerak/Documents/GitLab/risk-agent/data/experiments_results/{tuning_experiment}/'

tuning_experiments = [f for f in os.listdir(tuning_experiments_folder) if os.path.isdir(os.path.join(tuning_experiments_folder, f))]

In [5]:
tuning_experiments.sort()

In [6]:
tuning_experiments

['Paranoid_fortify_all_tuning_basic_evaluation_False',
 'Paranoid_fortify_all_tuning_basic_evaluation_True',
 'Paranoid_fortify_all_tuning_basic_heuristic_False',
 'Paranoid_fortify_all_tuning_basic_heuristic_True',
 'Paranoid_fortify_all_tuning_random_False',
 'Paranoid_fortify_all_tuning_random_True']

In [7]:
win_rates_all_experiments = {}

for experiment in tuning_experiments:
    data_path = f'/Users/moshalangerak/Documents/GitLab/risk-agent/data/experiments_results/{tuning_experiment}/{experiment}/game_level.csv'
    data = load_data(data_path)
    win_rates_per_player_type = calculate_win_rate_per_player_type(data)
    win_rates_all_experiments[experiment] = win_rates_per_player_type

In [8]:
win_rates_all_experiments_df = pd.DataFrame(win_rates_all_experiments).T

In [9]:
win_rates_all_experiments_df

,basic_evaluation,mcts,basic_heuristic,random
Paranoid_fortify_all_tuning_basic_evaluation_False,0.76,0.22,NaN,NaN
Paranoid_fortify_all_tuning_basic_evaluation_True,0.81,0.16,NaN,NaN
Paranoid_fortify_all_tuning_basic_heuristic_False,NaN,0.62,0.38,NaN
Paranoid_fortify_all_tuning_basic_heuristic_True,NaN,0.60,0.40,NaN
Paranoid_fortify_all_tuning_random_False,NaN,0.72,NaN,0.28
Paranoid_fortify_all_tuning_random_True,NaN,0.80,NaN,0.20
